<div style="text-align: center; margin: 50px">

<h1 style="color: black; background-color: grey; text-align: center;">Key Extension: Multi-Key FHE Utilizing LWR </h1>
<!-- <h3> A perspective to generalize NTRU variants </h3>
<h3>Prepared by: Ali Raya & Vikas Kumar </h3> -->
</div>

# Overview

[Our Construction](#intro) <br>
   * [Parameters](#params) <br>
   * [Key generation](#key) <br>
   * [Key extension](#epk) <br>
   * [Encryption](#enc) <br>
   * [Evaluation](#eval) <br>
   * [Decryption](#dec) <br>

<a id="intro"></a>
### Our Construction

Our (leveled) key extension multi-key fully homomorphic encryption scheme
consists of the following algorithms

$MKEFHE=\{SetUp,KeyGen,KeyExt,Enc,Eval,Dec\}.$

In [57]:
import math
import numpy as np
import random

In [58]:
def signed_mod(x, q):
    r = x % q
    return r - q if r > q / 2 else r

<a id="params"></a>
### Parameters

$\lambda$: security parameter

$d:$ circuit depth 

$PP←MKEFHE.SetUp(1^𝜆,1^𝑑):$

It takes as input the security
 parameter $\lambda$, circuit depth $𝑑$ and outputs the public parameters $PP= (𝑞,𝑝,𝑛,𝑚,𝑘,A)$, where 
 integer $𝑛=𝑛(\lambda,𝑑)$ be the lattice dimension, moduli $𝑞=𝑞(𝜆,𝑑),𝑝=𝑝(𝜆,𝑑)$ such that $𝑝|𝑞,𝑚=𝑂(𝑛log𝑞+log𝑝), 𝑙1=⌈log𝑞⌉$, $𝑙2=⌈log𝑝⌉, 𝑁=𝑛𝑙1+𝑙2, 𝑘 $ denotes the upper bound on the participants, and a uniformly chosen matrix $A∈\mathbb{Z}_q^{m\times n}$.

$n$: lattice dimension

$k$: upper bound on the number of parties

$N_p$: number of parties involved

$p,q$: two large moduli such that $q>>p$ and $p|q$





### To demonstrate the correctness of our scheme, in the following, we implement the toy example with dummy parameters:

In [59]:
n = 2**3
q = 2**16
p = 2**11
l1 = math.ceil(math.log2(q))
l2 = math.ceil(math.log2(p))
N = n*l1+l2
Np = 5 ### The number of the parties
m = n*l1+l2 ##big O

<a id="key"></a>
### Key Generation

#### The trusted third party first generates a common random matrix $A∈\mathbb{Z}_q^{m\times n}$ and shares with the parties involved

In [60]:
def generate_A(n,m,q):
    """
    Input:   -n,m: the dimension of the lattice
    Output:  A matrix A \in \Z_q^{m \times n}
    """
    return np.random.randint(0, q, size=(m, n))

In [61]:
A = generate_A(n,m,q)

In [62]:
A

array([[56028, 65455,  9095, ...,  2530, 33333, 19754],
       [37199, 32076,  4697, ..., 51976, 48060, 55769],
       [19057, 35819, 43840, ..., 45794, 55911,  2382],
       ...,
       [12068, 37366, 32305, ..., 16597,  8250, 40688],
       [23494, 35736, 16130, ..., 23149, 65498,   261],
       [56351, 63972, 64450, ..., 58249, 37506,  5082]])

### Secret and public key generation


$(pk_𝑗 ,sk_𝑗)←MKEFHE.KeyGen(PP):$ Let $𝑁_𝑝 ≤𝑘$ be the actual number of participants.The $𝑗𝑡ℎ$ party runs the key
 generation algorithm taking PP as input and outputs the keypair $(pk_𝑗 ,sk_𝑗)$. KeyGen randomly chooses a vector 
 
 $s_𝑗 = (𝑠_{𝑗,1},𝑠_{𝑗,2},...,𝑠_{𝑗,𝑛})∈\{0,1\}^𝑛$, 
 
 and computes
 
 $$b_{j}= {\lfloor{{<A, s_{j}>_q}}}\rceil_p = {\bigg\lfloor{{\frac{p}{q}<A, s_{j}>_q}\bigg\rceil} \mod p \in \mathbb{Z}_{p}^{m}}.$$
 
 The public key of the party 
 
 $$pk_{j}= B_{j}= [b_{j} ~~~ A]  \in \mathbb{Z}_p^{m\times 1} [\times]\mathbb{Z}_{q}^{m \times n},$$
 
and secret key is 

$$\displaystyle sk_{j}=\bigg(1 ~\bigg|~ -\frac{p}{q}s_{j}\bigg).$$

In [63]:
def si(n):
    """
    Input: -n: the length of the sequence
    Output: a binary sequence of length n 
    """
    return np.random.randint(0, 2, size=n)

In [64]:
S = []
for i in range(Np):
    S.append(si(n))

In [65]:
# s0 = si(n)
# s1 = si(n)
# s2 = si(n)

In [66]:
S

[array([1, 0, 0, 1, 1, 1, 0, 1]),
 array([1, 1, 0, 0, 1, 1, 1, 0]),
 array([1, 1, 1, 0, 1, 0, 0, 0]),
 array([1, 1, 1, 1, 1, 1, 0, 0]),
 array([1, 0, 1, 1, 0, 0, 0, 1])]

In [67]:
def bi(A,s,p,q,m):
    """
    Input: - A: a random matrix generated by the trusted party
           - s: a private sequence represents the private secret
           - p and q: two modulus
           - m: the number of the rows in A
    """
    
    As = np.matmul(A,s)
    As = As.astype(float)
    for i in range(m):
        As[i] = signed_mod(round(p*As[i]/q),p)
        
    return As

In [68]:
B = []
for i in range(Np):
    B.append(bi(A,S[i],p,q,m))

In [69]:
def ski(si, p, q,n):
    """
    Input: -si: the private binary sequence
           -p,q: two modulus
           -n: the length of si
    """
    ski = [0]*n
    for i in range(n):
        ski[i] = float((-p*si[i])/q)
    ski = [1]+ ski
    
    return ski

In [70]:
SK = []
for i in range(Np):
    SK.append(np.array(ski(S[i],p,q,n)))
    

In [71]:
b_c = []
for i in range(Np):
    b_c.append(B[i].reshape(-1,1))

In [72]:
PK = []
for i in range(Np):
    PK.append(np.hstack((b_c[i], A)))

In [73]:
# for i in range(m):
#     for j in range(n+1):
#         bki[i,j] = int(bki[i,j])

In [74]:
Res = []
for i in range(Np):
    Res.append( np.matmul(PK[i],SK[i].reshape(-1,1)))

In [75]:
for i in range(Np):
    for j in range(m):
        Res[i][j] = signed_mod(Res[i][j],p)

## Check

The relationship between the public key and the secret key is 



$$({pk}_j \cdot sk_j^T)~ { mod } p = (b_j - \frac{p}{q} A \cdot s_j^T){ ~mod } p = e_j \in [-1/2, 1/2]^m.$$

In [76]:
for i in range(Np):
    for j in range(m):
#         print(abs(Res[i][j]))
        assert(abs(Res[i][j])<=0.5)

## Key Extension

$epk \leftarrow MKEFHE.KeyExt(\{pk_{j}\}_{j\in [N_p]})$: It takes as input the public key of all the parties and outputs the extension key 

$$epk = B = [\bar{b} ~~~ A]  \in \mathbb{Z}_p^{m\times 1}[\times]\mathbb{Z}_q^{m\times n},$$ 

where $\displaystyle \bar{b}=\sum_{j=0}^{N_p}b_{j} \mod p$.

The corresponding secret key $\displaystyle sk = \bigg(1 ~\bigg|~ -\frac{p}{q} \bar{s}\bigg)$, where $\displaystyle\bar{s}=\sum_{j=1}^{N_p}s_{j}.$ 

In [77]:
bbar = [0]*m

for i in range(Np):
    for j in range(m):
         bbar[j]+= B[i][j] 
print(bbar)           
for i in range(m):
    bbar[i] = signed_mod(bbar[i],p)
print(bbar)
    
Bbar = np.hstack((np.array(bbar).reshape(-1,1), A))

[299.0, -845.0, 832.0, -942.0, 1039.0, 901.0, -1195.0, -420.0, -1348.0, -440.0, 67.0, -108.0, 839.0, 224.0, 1504.0, -2539.0, -705.0, 2353.0, -892.0, 1384.0, 171.0, -94.0, 449.0, -1087.0, 1150.0, 1325.0, -2637.0, -1741.0, 94.0, 570.0, 180.0, 2183.0, 640.0, 1229.0, -648.0, -226.0, 863.0, 829.0, 237.0, 378.0, 794.0, -609.0, 1358.0, 2714.0, 1479.0, 425.0, -536.0, -315.0, 974.0, 396.0, -613.0, -1591.0, -1678.0, 337.0, 1088.0, -2105.0, 1781.0, -68.0, -663.0, 1040.0, -3049.0, 659.0, 1237.0, 1770.0, 442.0, -459.0, -49.0, -2955.0, 2412.0, -1189.0, -1063.0, 845.0, 1691.0, 914.0, -1184.0, -460.0, 837.0, -341.0, 456.0, -1261.0, -51.0, -2710.0, -721.0, 709.0, -230.0, -1703.0, -392.0, 1130.0, 697.0, 406.0, -626.0, 18.0, -177.0, 3177.0, 536.0, -1395.0, 2037.0, 1155.0, 1630.0, -1456.0, 1009.0, -214.0, 269.0, 1721.0, 285.0, -937.0, -876.0, -113.0, 2300.0, 122.0, -62.0, 1961.0, 1207.0, 302.0, 2534.0, -853.0, 44.0, -483.0, 905.0, -1985.0, -3545.0, -1276.0, 1494.0, 1140.0, 506.0, -1203.0, 24.0, 1106.0, -8

In [78]:
sbar = [0]*n

for i in range(Np):
    for j in range(n):
         sbar[j]+= S[i][j] 

print(sbar)
for i in range(n):
    sbar[i] = (-p*sbar[i])/(q)

    
SKbar = [1]+sbar

[5, 3, 3, 3, 4, 3, 1, 2]


In [79]:
ress = np.matmul(Bbar, SKbar)

In [80]:
for i in ress:
    print(abs(signed_mod(i,p)))

0.28125
0.53125
0.59375
0.21875
0.34375
1.21875
0.09375
0.03125
0.3125
1.375
0.15625
0.78125
0.125
0.90625
0.3125
1.09375
0.40625
0.40625
0.59375
0.25
0.84375
0.34375
0.5625
0.25
0.03125
0.625
0.78125
0.0
0.71875
0.0625
0.46875
0.65625
0.1875
0.59375
1.0625
0.0
0.21875
1.59375
0.15625
0.09375
0.0625
0.71875
0.21875
0.75
0.125
0.34375
0.375
0.78125
0.65625
0.65625
0.15625
1.0
0.3125
0.0
0.6875
0.625
0.03125
0.65625
0.25
0.03125
0.5
0.875
0.9375
0.3125
0.84375
0.5
0.0625
1.0625
1.125
0.65625
0.0625
0.625
0.1875
0.15625
0.21875
0.03125
0.53125
0.34375
0.4375
0.5625
0.53125
0.1875
0.5
0.28125
0.3125
0.875
0.4375
0.53125
1.03125
0.46875
0.09375
0.375
0.0625
0.71875
1.25
0.46875
0.59375
0.125
0.1875
1.03125
1.1875
0.25
0.09375
0.9375
0.71875
0.1875
0.5
0.875
0.84375
0.625
0.375
0.40625
0.6875
0.90625
0.875
0.125
0.3125
0.59375
1.5625
0.125
0.6875
0.15625
0.75
0.34375
0.28125
0.40625
0.3125
0.125
0.125
0.59375
0.4375
0.125
0.90625
0.5625
0.8125
0.0
0.0
0.40625
0.84375


In [81]:
for i in ress:
    assert(abs(signed_mod(i,p))<=Np*0.5)

<a id="enc"></a>
### Encryption


$C_{j} \leftarrow MKEFHE.Enc(epk, \mu_{j})$: To generate a ciphertext corresponding to a message $\mu_{j}\in \{0, 1\}$ under the extension key $epk = B$, the $j^{th}$ party computes the ciphertext $C_{j}$ as \begin{equation}\label{fresh_cipher}
        C_{j}=\mu_{j}G + B^{T}R_{j} \in \mathbb{Z}_p^{1\times N} [\times] \mathbb{Z}_q^{n \times N}, 
        \end{equation}
        
 where $R_j \in \{0, 1\}^{m \times N}$ is a uniformly chosen binary matrix and $G \in \mathbb{Z}_p^{1\times N} [\times] \mathbb{Z}_q^{n \times N}$ is the Gadget matrix

In [82]:
def gadget_matrix(l1, l2, n, N):
    """
    Input: - l1 = math.ceil(math.log2(q))
           - l2 = math.ceil(math.log2(p))
           - N = n*l1+l2
    Output: the gadget matrix (n+1 * N)
    G = [g2         \\g1         \\        g1]
    First row of the matrix if modulo p and other rows are modulo q
    """
    g1 = []
    g2 = []
    
    for i in range(l1):
        g1.append(2**i)
    
    for i in range(l2):
        g2.append(2**i)
        
            
#     for i in range(l1-1,-1,-1):
#         g1.append(2**i)
    
#     for i in range(l2-1,-1,-1):
#         g2.append(2**i)
        
    zerol2 = [0]*l2
    zerol1 = [0]*l1
    
    rows, cols = n+1, N  # n+1 rows, N columns
    G = [[[] for _ in range(cols)] for _ in range(rows)]
    
    ## Fill first row of G
    for i in range(l2):
        G[0][i] = g2[i]
    for i in range(l2,N):
        G[0][i] = 0
        
    ### Fill other rows
    for i in range(1,rows):
        for j in range(cols):
            if(j>=l2+(i-1)*l1 and j<l2+i*l1):
                G[i][j]=g1[j-(l2+(i-1)*l1)]
            else:
                G[i][j] = 0
    return np.array(G)
                
    
    
    
    
    

In [83]:
G =gadget_matrix(l1, l2, n, N)

In [84]:
Rj = generate_A(m,N,2)
Muj = np.random.randint(0,2)
# Muj = 0
Cj = Muj*G+ np.matmul(Bbar.transpose(), Rj)

for i in range(N):
    Cj[0][i] = signed_mod(Cj[0][i],p)
    
for i in range(1,n+1):
    for j in range(N):
        Cj[i][j] = signed_mod(Cj[i][j],q)

<a id="dec"></a>
### Decryption

$\mu \leftarrow {MKEFHE.Dec((sk_{1}, \ldots, sk_{N_p}), {C^*})}$: Decryption is a multi-party protocol. It takes as input the secret key of all the parties, a ciphertext (either fresh or evaluated), and outputs the corresponding plaintext message. The algorithm takes place in two steps:


#### 1. Partial decryption

Let ${C^*}=\left[\begin{smallmatrix}
				{c}_{0}^*\\
				{C}_{1}^* 
			\end{smallmatrix} \right]\in  \mathbb{Z}_p^{1\times N} [\times] \mathbb{Z}_q^{n \times N}$, where ${c}_{0}^*\in  \mathbb{Z}_p^{1\times N} $ and ${C}_{1}^* \in \mathbb{Z}_q^{n \times N}.$ 
            
            
Each party computes

\begin{equation}\label{part_dec}
    p_{i}=s_{i}{C}_{1}^*G^{-1}(w^{T}) + e_i^{sn},
\end{equation}

where $w = [\lceil p/2 \rceil, 0, \ldots, 0] \in \mathbb{Z}_p [\times] \mathbb{Z}_q^n$ and $e_i^{sn} \in [-B_{sn}, B_{sn}]$ is called as smudging noise, where $B_{sn} = 2^{O(\lambda d \log{\lambda})} $.

In [85]:
c0s = Cj[0][:]
c1s = Cj[1:][:]

In [86]:
def Ginv(w,l2,l1,n):
    """
    Input: - p,q: the modulus
           - n: the lattice dimension
           - w: a list to be expanded
           
    The function represent the first coefficient of w with respect to p
    and the other coefficients with respect to q
    """
    bs = format(w[0], f'0{l2}b')
    bs = bs[::-1]
    for i in range(1,n+1):
        bs+=format(w[i], f'0{l1}b')[::-1]
    return np.array(list(bs), dtype=int)

In [87]:
def Ginv_mat(mat,l2,l1,n,m):
    """
    Input: - p,q: the modulus
           - n: the lattice dimension
           - m: the parameter m
           - w: a list to be expanded
           
    Similar to the Ginv fuction but acting on matrices rather than lists,
    the function represents the first coefficient of w with respect to p
    and the other coefficients with respect to q
    """
    out = np.zeros((m, m), dtype=int) 
    for col in range(m):
        w = mat[:,col]
        bs = format(int(w[0]), f'0{l2}b')
        bs = bs[::-1]
        for i in range(1,n+1):
            bs+=format(int(w[i]), f'0{l1}b')[::-1]
        newcol= np.array(list(bs), dtype=int)
        out[:,col] = newcol
    return out

In [88]:
w = [math.ceil(p/2)]+[0]*n

In [89]:
ginv = Ginv(w,l2,l1,n)

In [90]:
def partial_decryption(si,c1s, w, ei,l1,l2,n):
    """
    Input: -si: one party secret key
           -c1s: the second part of the ciphertext matrix
           -w: [ceil(p/2),0,0,....0]
           -ei: an error
    """
    t0 = Ginv(w,l2,l1,n)
    t0 = t0.transpose()
   
    t1 = np.matmul(si,c1s)
#     print(t1)
#     print(t0)
    t1 = np.matmul(t1,t0)
#     print(t1)
    ei = random.randint(-ei, ei)
    return t1+ei, ei
   

In [91]:
pis = []
esn = []

for i in range(Np):
    pi, ei = partial_decryption(S[i], c1s, w, 1, l1,l2,n)
    pis.append(pi)
    esn.append(ei)

#### 2. Final decryption

 The final decryption follows as:
 
  \begin{equation}\label{v_com}
             \displaystyle v = {c}_0^*G^{-1}(w^{T}) -\frac{p}{q}\sum_{i=1}^{N_p}p_i \mod{p}.
         \end{equation} 
         
  Then, \begin{equation}
         \displaystyle \mu = \bigg\lfloor{\frac{v}{\lceil p/2 \rceil}}\bigg\rceil.   
         \end{equation}

In [92]:
def final_decryption(c0s, w, p, q, Np, l1, l2,n, pis):
    """
    Input: - c0s: the first row of the ciphertex
           - w: [ceil(p/2),0,0,....0]
           - p, q: modulus
           - Np: the number of the parties
    """
    t0 = Ginv(w,l2,l1,n)
#     print("t0: ", t0)
#     print(t0.shape)
    t1 = np.matmul(c0s, t0.transpose())
#     print("t1: ", t1)
    t2 = 0
    for i in range(Np):
        t2 = t2+pis[i]
#     t2 = signed_mod(t2,q)
    t2 = (p/q)*t2
#     print("t2: ", t2)
#     print("t1-t2: ", t1-t2)
    v = signed_mod((t1-t2),p) 
#     print("v is : ", v)
#     print(v/(math.ceil(p/2)))
    mu = round(v/(math.ceil(p/2)))
    return abs(mu)



In [93]:
mu=final_decryption(c0s, w, p, q, Np, l1, l2, n , pis)

In [94]:
print(Muj)
print(mu)

1
1


## Test it for many trials

In [95]:
trial = 0  

while(trial<1000): 
    Rj = generate_A(m,N,2)
    Muj = np.random.randint(0,2)
#         Muj = 0
    Cj = Muj*G+ np.matmul(Bbar.transpose(), Rj)

    for i in range(N):
        Cj[0][i] = Cj[0][i]%p

    for i in range(1,n+1):
        for j in range(N):
            Cj[i][j] = Cj[i][j]%q


    c0s = Cj[0][:]
    c1s = Cj[1:][:]


    w = [math.ceil(p/2)]+[0]*n
    pis = []
    esn = []
    for i in range(Np):
        pi, ei = partial_decryption(S[i], c1s, w, 1, l1,l2,n)
        pis.append(pi)
        esn.append(ei)
    
    mu=final_decryption(c0s, w, p, q, Np, l1, l2, n , pis)
    print("Original message: ", Muj)
    print("Decrypted message:", mu)
    assert(Muj==mu)
    print("--------------------------------")
    trial +=1


print("All decrypted successfully!")


Original message:  1
Decrypted message: 1
--------------------------------
Original message:  0
Decrypted message: 0
--------------------------------
Original message:  0
Decrypted message: 0
--------------------------------
Original message:  1
Decrypted message: 1
--------------------------------
Original message:  1
Decrypted message: 1
--------------------------------
Original message:  0
Decrypted message: 0
--------------------------------
Original message:  0
Decrypted message: 0
--------------------------------
Original message:  1
Decrypted message: 1
--------------------------------
Original message:  0
Decrypted message: 0
--------------------------------
Original message:  1
Decrypted message: 1
--------------------------------
Original message:  0
Decrypted message: 0
--------------------------------
Original message:  1
Decrypted message: 1
--------------------------------
Original message:  1
Decrypted message: 1
--------------------------------
Original message:  0
Decr

<a id="eval"></a>
## Evaluated ciphertext

### Testing on evalutated ciphertext

 $\widehat{C} \leftarrow  MKEFHE.Eval(epk, \mathcal{F}, ({C}_{1}, \ldots, {C}_{r}))$:  It takes as input the extension key $epk$, a tuple of ciphertexts $({C}_{1}, \ldots, {C}_{r})$, a circuit $\mathcal{F}$ and outputs an evaluated ciphertext $ \widehat{C}$.
 
 
 For further discussion, we considered the scenario of two parties. Let $C_{1} = MKEFHE.Enc(epk, \mu_{1}) = \mu_{1}G + B^TR_1,{{~~ and ~~~}} C_{2} = MKEFHE.Enc(epk, \mu_{2}) = \mu_{2}G + B^T R_2$ be two fresh ciphertexts.

In [96]:
### First message
R0 = generate_A(m,N,2)
M0 = np.random.randint(0,2)
#         Muj = 0
C0 = M0*G+ np.matmul(Bbar.transpose(), R0)

for i in range(N):
    C0[0][i] = C0[0][i]%p

for i in range(1,n+1):
    for j in range(N):
        C0[i][j] = C0[i][j]%q


### Second message
R1 = generate_A(m,N,2)
M1 = np.random.randint(0,2)
#         Muj = 0
C1 = M1*G+ np.matmul(Bbar.transpose(), R1)
for i in range(N):
    C1[0][i] = C1[0][i]%p

for i in range(1,n+1):
    for j in range(N):
        C1[i][j] = C1[i][j]%q


## Multi-Key FHE: Addition


$C_{\sf add} \leftarrow MKEFHE.add(C_{1}, C_{2})$: Input ciphertexts $C_{1}, C_{2}$, and outputs $C_{\sf add}$, calculated as

  \begin{eqnarray}
                    C_{add} &=& C_{1} + C_{2}\nonumber\\
                &=& (\mu_{1}+\mu_{2})G+B^{T}(R_{1}+R_{2}),
                \end{eqnarray}
                where 
               
$$C_{add}\in \mathbb{Z}_p^{1\times N} [\times] \mathbb{Z}_q^{n \times N}.$$

In [97]:
Cadd = C0+C1
for i in range(N):
    C1[0][i] = C1[0][i]%p

for i in range(1,n+1):
    for j in range(N):
        C1[i][j] = C1[i][j]%q


In [98]:
c0s = Cadd[0][:]
c1s = Cadd[1:][:]

w = [math.ceil(p/2)]+[0]*n

pis = []
esn = []
for i in range(Np):
    pi, ei = partial_decryption(S[i], c1s, w, 1, l1,l2,n)
    pis.append(pi)
    esn.append(ei)
        
mu=final_decryption(c0s, w, p, q, Np, l1, l2, n , pis)
print("Original message: ", M0^M1)
print("Decrypted message:", mu)

assert(M0^M1 == mu)

Original message:  0
Decrypted message: 0


## Multi-Key FHE: Multiplication

$C_{\sf mult} \leftarrow MKEFHE.mult(C_{1}, C_{2})$: Input ciphertexts $C_{1}, C_{2}$, and outputs $C_{mult}$, calculated as 

\begin{align}\label{mult1}
				C_{\sf mult} &= C_{1}G^{-1}(C_{2})\nonumber \\
                &= \mu_{1}\mu_{2}G + B^{T}(\mu_1R_{2} + R_{1}G^{-1}(C_{2}))
  \end{align}
  
  where 
  
$$C_{mult} \in \mathbb{Z}_p^{1\times N} [\times] \mathbb{Z}_q^{n \times N}.$$

In [99]:
GinvC1 = Ginv_mat(C1,l2,l1,n,m)
Cmul = np.matmul(C0,GinvC1)

In [100]:
c0s = Cmul[0][:]
c1s = Cmul[1:][:]

w = [math.ceil(p/2)]+[0]*n

pis = []
esn = []
for i in range(Np):
    pi, ei = partial_decryption(S[i], c1s, w, 1, l1,l2,n)
    pis.append(pi)
    esn.append(ei)
mu=final_decryption(c0s, w, p, q, Np, l1, l2, n , pis)
print("Original message: ", M0*M1)
print("Decrypted message:", mu)
assert(M0*M1 == mu)

Original message:  1
Decrypted message: 1


## Multi-Key FHE: One  NAND gate 

$C_{\sf mult} \leftarrow MKEFHE.NAND(C_{1}, C_{2})$: Input ciphertexts $C_{1}, C_{2}$, and outputs $C_{NAND}$, calculated as 

$$C_{NAND} = G - C_{1}G^{-1}(C_{2}) \in \mathbb{Z}_p^{1\times N} [\times] \mathbb{Z}_q^{n \times N}.$$

In [101]:
def logical_not(x: int) -> int:
    return 1 if x == 0 else 0

def NAND(a: int, b: int) -> int:
    return logical_not(a & b)

In [102]:
GinvC1 = Ginv_mat(C1,l2,l1,n,m)
Cmul = np.matmul(C0,GinvC1)
CNAND = G-Cmul

In [103]:
c0s = CNAND[0][:]
c1s = CNAND[1:][:]

w = [math.ceil(p/2)]+[0]*n

pis = []
esn = []
for i in range(Np):
    pi, ei = partial_decryption(S[i], c1s, w, 1, l1,l2,n)
    pis.append(pi)
    esn.append(ei)
mu=final_decryption(c0s, w, p, q, Np, l1, l2, n , pis)
print("Original message: ", NAND(M0, M1))
print("Decrypted message:", mu)
assert(NAND(M0, M1) == mu)

Original message:  0
Decrypted message: 0
